In [27]:
from functools import wraps

All in all, we can say that a decorator in Python is a callable Python object that is used to modify a function, method or class definition

let's modify the function sin to rounded to 2 numbers

<code>@g
def f<code>

is equivalent to <code>f = g(f)</code>

Modify functions that are already available(from modules, ...)

In [6]:
import numpy as np
def our_decorator(func):
    def wrapper(x):
        return round(func(x), 2)
    return wrapper

np.sin = our_decorator(np.sin)

np.sin(np.pi / 3)

0.87

Modify self-created functions

In [18]:
def decorator(func):
    def wrapper(name):
        print('Hello')
        print(func(name))
    return wrapper
@decorator
def greeting(name):
    return name

In [19]:
greeting('VN Pikachu')

Hello
VN Pikachu


How this works?<br>
```python
greeting = decorator(greeting)
```

In [41]:
def my_decorator(value):
    def wrapper(name): #wrapper(func)
        print('hello')
        print(value)
        print(name)
        def inner_wrapper(name):
            print(name)
        return inner_wrapper
    return wrapper
    
@my_decorator('VN-champions')
def my_greeting(name):
    print(name)

my_greeting('VN Pikachu')

hello
VN-champions
<function my_greeting at 0x000001B05DA94BF8>
VN Pikachu


In [44]:
def decorator(func):
    def wrapper(name):
        func(name)
    return wrapper
def outer_decorator(value):
    print(value)
    return decorator
#we can uderstand @outer_decorator('VN Champions')
#will call function outer_decorator('VN Champions')
#then outer_decorator('VN Champions') return function decorator
#so @outer_decorator('VN Champions') is turned to @decorator
#note that decorator is a function
#it means @function
#or @call_a_function_and_return_another_function
@outer_decorator('VN Champions') 
def greet(name):
    print(name)
    
greet('VN Pikachu')

VN Champions
VN Pikachu


```python
def decorator(func): #this is the function that we want to modify
    def wrapper(*kwargs): #we we call our target function, it's values we pass will transfer to kwargs
        #...... some code here
    return wrapper
@decorator
def target_function(*kwargs):
    #.... some code here
```

# Use cases for Decorators

## Checking Arguments

In [1]:
def decorator(func):
    def wrapper(x):
        if type(x) == int and x >= 0:
            return func(x)
        else:
            raise Exception('Invaid input')
    return wrapper
@decorator
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)

In [2]:
factorial(5)

120

In [3]:
factorial(-1)

Exception: Invaid input

## Counting the number of function calls

In [8]:
def decorator(func):
    def wrapper(*args, **kwargs):
        wrapper.counter += 1
        return func(*args, **kwargs)
    wrapper.counter = 0
    return wrapper
@decorator
def do_nothing(*arg, **kwargs):
    pass


In [9]:
do_nothing()
do_nothing(3, 4, 2)
do_nothing(x = 2, y = 3)

In [10]:
do_nothing.counter

3

## Decorator with parameters

In [11]:
def decorator_wrapper(txt):
    def decorator(func):
        def wrapper(x):
            print(txt)
            func(x)
        return wrapper
    return decorator
@decorator_wrapper('decorator with parameters')
def print_a_number(x):
    print(f'number {x}')


In [12]:
print_a_number(10)

decorator with parameters
number 10


## Saving the name, docstring and module of a function when using Decorator

In [16]:
def decorator(func):
    def wrapper(x):
        '''this is the docstring of wrapper'''
        print('This is an example of decorator')
        func(x)
    return wrapper
@decorator
def greeting(name):
    print(f'Hello {name}')

In [17]:
greeting('VN Pikachu')

This is an example of decorator
Hello VN Pikachu


In [18]:
greeting.__name__

'wrapper'

In [19]:
greeting.__doc__

'this is the docstring of wrapper'

We can see that afer using decorator on the function <code>greeting</code>, the name of the function and the docstring of is replaced by the information provided by the <code>wrapper</code>

Solution: Using <code>wraps</code> from <code>functools</code>

In [23]:
from functools import wraps
def decorator(func):
    @wraps(func)
    def wrapper(name):
        print('decorator with wraps')
        func(name)
    return wrapper
@decorator
def greeting(name):
    'This is the docstring of the function greeting'
    print(f'hello {name}')

In [22]:
greeting('Tank Cao')

decorator with wraps
hello Tank Cao


In [24]:
greeting.__name__

'greeting'

In [25]:
greeting.__doc__

'This is the docstring of the function greeting'

## Classes instead of Functions

In [4]:

class Polynomial:
    '''
        Create a 2 order Polynomial
        
    '''
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    #return the result of ax^2 + bx + c
    def __call__(self, x): 
        return self.a * x * x + self.b * x + self.c

In [7]:
f = Polynomial(1,-2,1)  #x^2 + 2x + 1

In [8]:
#evaluate at x = 1
f(1)

0

<hr>

Decorate with a function:

In [20]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print('Decorating....')
        func(*args, **kwargs)
    return wrapper

@decorator
def greeting(name):
    print(name)
    
greeting('Hello World')

Decorating....
Hello World


Decorate with a class (the normal way):

In [24]:
class decorator:
    def __init__(self, func):
        self.function = func
    def __call__(self, *args, **kwargs):
        print('Decorating....')
        self.function(*args, **kwargs)
        
def greeting(name):
    print(name)
    
greeting = decorator(greeting) #instatiation, greeting now is an instance of class `decorator`, not a function anymore

greeting('Hello World')

Decorating....
Hello World


Equivalently, decorate with a class (via **`__init__`**) (The pythonic wya)

In [28]:
class decorator:
    def __init__(self, func):
        self.function = func
    def __call__(self, *args, **kwargs): 
        print('Decorating....')
        self.function(*args, *kwargs)
    
@decorator  #call __init__, receive a function, return self (an instance of class decorator)
#greeting will be transformed from a function to an instance of class decorator
def greeting(name):
    print(name)
    
greeting('Hello world')

Decorating....
Hello world


<hr>

Decorator with an instance of a class( via **`__call__`**):

In [5]:
class BOT:
    def __init__(self, message):
        self.message = message
    def __call__(self, func):
        def wrapper(name):
            print(self.message)
            func(name)
        return wrapper


In [6]:
@BOT('FUCK YOU') #This return function __call__ to decorate
def greet(name):
    print(f'Hello, {name}')

In [7]:
greet('Hadi')

FUCK YOU
Hello, Hadi


**Exercise**: Implement a cache decorator for function that calculate the n-th Fibonacci

In [33]:
class Cache:
    def __init__(self, func):
        self.function = func
        self.memo = {0:0, 1:1}
    def __call__(self, n):
        if n in self.memo:
            return self.memo[n]
        res = self.function(n)
        self.memo[n] = res
        return res

In [38]:
@Cache
def fib(n):
    """
    This function calculate the n-th Fibonacci number
    """
    return fib(n - 1) + fib(n - 2)

fib(0), fib(50), fib(3), fib(4), fib(5), fib(6)

(0, 12586269025, 2, 3, 5, 8)

# Example

Let’s say we want to print a deprecation warning on stderr on the first invocation of a function we don’t like anymore. If we don’t want to modify the function, we can use a decorator:

In [9]:
#Decorate using class
class deprecation:
    def __call__(self, func):
        self.func = func
        self.count = 0
        return self.wrapper
    def wrapper(self,*args):
        if self.count == 0:
            print('Deprecated Function')
        self.count += 1
        return self.func(*args)

In [10]:
@deprecation()
def add(a, b):
    return a + b

In [11]:
add(1,3)

Deprecated Function


4

In [12]:
add(5,3)

8

In [40]:
#Decorate using Function
def OUTDATED(func):
    def wrapper(*args):
        if  not wrapper.count:
            print('Outdated Function')
        wrapper.count += 1
        return func(*args)
    wrapper.count = 0
    return wrapper
        

In [41]:
@OUTDATED
def multiply(a,b):
    return a * b

In [42]:
multiply(1,3)

Outdated Function


3

In [43]:
multiply(3,3)

9